In [19]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# Key Question : How does the demographic and the income level of the people affect the preference and acceptance of self-driving cars? 

### Preferences for self-driving cars by age, gender, education, and income levels
### Correlations between these variables

In [20]:
#Read raw file
df=pd.read_csv('https://raw.githubusercontent.com/EvaW01/schulich_ai_fundamentals/main/MMAI_Fundamental_Assignment.csv')

df.head()


,Unnamed: 0,Question_key,Interview_start_time,Interview_end_time,Device_Type,Language,Form_No,Tablet_category,Read_about_SelfDrive_Car,Widespread_use_SelfDrive_Car,...,People_Political_Party,People_Political_Type,People_Political_Type_Category,Income_Level,Vote_Registered_Adress,Ideology,Interaction_Frequency,Volume_Sum,Income_Class,Weight_99_Percent
0,0,100260.0,2021-11-03 14:25:27,2021-11-03 14:45:34,Laptop/PC,English,Form 2,Non-tablet HH,NaN,NaN,...,NaN,Rep/Lean Rep,Conservative Rep/Lean,"$50,000 to less than $60,000",You are ABSOLUTELY CERTAIN that you are regist...,Very conservative,Several times a day,No,Middle income,0.206396
1,1,100314.0,2021-11-04 12:35:35,2021-11-04 12:55:29,Smartphone,English,Form 1,Non-tablet HH,A little,Not sure,...,NaN,Rep/Lean Rep,Moderate/Liberal Rep/Lean,"$40,000 to less than $50,000",You are ABSOLUTELY CERTAIN that you are regist...,Liberal,Several times a day,Yes,Middle income,0.315090
2,2,100363.0,2021-11-03 20:23:43,2021-11-03 20:36:24,Smartphone,English,Form 1,Non-tablet HH,A little,Good idea for society,...,NaN,Dem/Lean Dem,Moderate/Conservative Dem/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are regist...,Moderate,Several times a day,No,Upper income,0.829579
3,3,100598.0,2021-11-02 13:01:05,2021-11-04 12:37:42,Laptop/PC,English,Form 2,Non-tablet HH,NaN,NaN,...,NaN,Rep/Lean Rep,Conservative Rep/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are regist...,Conservative,Several times a day,Yes,Upper income,0.337527
4,4,100637.0,2021-11-02 12:32:58,2021-11-02 12:46:23,Laptop/PC,English,Form 2,Non-tablet HH,NaN,NaN,...,The Republican Party,Rep/Lean Rep,Conservative Rep/Lean,"$30,000 to less than $40,000",You are ABSOLUTELY CERTAIN that you are regist...,Very conservative,Less often,No,Lower income,1.210606


In [21]:
unique_ages = df['Age'].unique()
unique_ages

array(['65+', '30-49', '50-64', '18-29', 'Refused'], dtype=object)

In [8]:
from scipy.stats import chi2_contingency

In [10]:
# Function to perform chi-square test and print the result
def chi_square_test(col1, col2):
    contingency_table = pd.crosstab(df[col1], df[col2])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    print(f"Chi-square test between {col1} and {col2}:")
    print(f"Chi2: {chi2}, p-value: {p}, Degrees of freedom: {dof}")
    print("Expected frequencies:")
    print(expected)
    print("\n")

# Perform chi-square tests for relevant pairs
chi_square_test('Gender', 'Read_about_SelfDrive_Car')
chi_square_test('Gender', 'Widespread_use_SelfDrive_Car')
chi_square_test('Education', 'Read_about_SelfDrive_Car')
chi_square_test('Education', 'Widespread_use_SelfDrive_Car')
chi_square_test('Income_Level', 'Read_about_SelfDrive_Car')
chi_square_test('Income_Level', 'Widespread_use_SelfDrive_Car')
chi_square_test('Age', 'Read_about_SelfDrive_Car')
chi_square_test('Age', 'Widespread_use_SelfDrive_Car')


Chi-square test between Gender and Read_about_SelfDrive_Car:
Chi2: 411.61390942965164, p-value: 4.55044222128367e-83, Degrees of freedom: 9
Expected frequencies:
[[1.44270328e+03 6.29943722e+02 1.93388317e+02 3.96468077e+00]
 [1.80496798e+03 7.88123423e+02 2.41948380e+02 4.96021735e+00]
 [1.97021153e+01 8.60275568e+00 2.64098583e+00 5.41432175e-02]
 [7.62662527e+00 3.33009897e+00 1.02231710e+00 2.09586649e-02]]


Chi-square test between Gender and Widespread_use_SelfDrive_Car:
Chi2: 238.5680712968734, p-value: 2.5752396757505966e-46, Degrees of freedom: 9
Expected frequencies:
[[1.00879100e+03 6.37432564e+02 6.18490200e+02 5.28624102e+00]
 [1.26209975e+03 7.97492723e+02 7.73793906e+02 6.61362313e+00]
 [1.37764409e+01 8.70502620e+00 8.44634194e+00 7.21909567e-02]
 [5.33281584e+00 3.36968756e+00 3.26955172e+00 2.79448865e-02]]


Chi-square test between Education and Read_about_SelfDrive_Car:
Chi2: 200.30213444489345, p-value: 2.8634133665196195e-38, Degrees of freedom: 9
Expected frequen